In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pickle

In [2]:
dataset_name = "prostate_cancer"
#dataset_name = "animal_faces"
#model_from = "task1"
model_from = "pretrained"

In [4]:
dataset_url = "/content/drive/MyDrive/MLProject/Codes/Task2/Features/features_labels_"+model_from+"_model_"+dataset_name+".csv"
dataset = pd.read_csv(dataset_url)

In [5]:
print(dataset.shape)
print(dataset.head)

(6000, 513)
<bound method NDFrame.head of              0         1         2         3         4         5         6  \
0     1.438999  0.078806  1.141035  1.938414  0.488909  0.426002  0.445668   
1     1.546778  0.052872  0.288157  0.307745  2.364518  0.513626  0.104222   
2     0.585797  0.199779  0.008568  2.106367  0.872578  0.000000  0.083597   
3     0.608120  0.112737  0.016985  0.914887  1.424414  0.166883  0.160807   
4     0.502600  0.051433  0.294563  0.527870  1.450859  0.105591  0.148526   
...        ...       ...       ...       ...       ...       ...       ...   
5995  0.122367  0.072377  0.385997  0.924579  0.890233  0.001040  0.265616   
5996  0.208139  0.314666  0.006673  0.770476  1.734351  0.201996  0.008273   
5997  1.909944  0.328913  0.238146  1.136451  1.832376  1.241467  0.187652   
5998  0.933171  0.057718  0.000000  0.788418  0.719934  0.530506  0.048958   
5999  1.210402  0.024202  0.475783  1.112730  0.748531  0.589873  0.116697   

             7       

In [6]:
# Separate features and labels
X = dataset.iloc[:, :-1]  # Features (all columns except the last one)
y = dataset.iloc[:, -1]   # Labels (last column)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
rf_classifier = RandomForestClassifier()

In [9]:
# Define hyperparameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

In [10]:
grid_search = GridSearchCV(rf_classifier, param_grid, cv=10, scoring='accuracy',return_train_score=True)
grid_search.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
train_accuracy = grid_search.cv_results_['mean_train_score']
test_accuracy = grid_search.cv_results_['mean_test_score']

In [ ]:
grid_search_cv_results = pd.DataFrame(columns=['hyperparameters', 'train_accuracy', 'test_accuracy'])

In [ ]:
for params, train_acc, test_acc in zip(grid_search.cv_results_['params'], train_accuracy, test_accuracy):
    print(f"Model with Hyperparameters: {params}")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Testing Accuracy: {test_acc:.4f}\n")
    new_row = pd.Series({'hyperparameters': params, 'train_accuracy': train_acc, 'test_accuracy': test_acc})
    grid_search_cv_results = grid_search_cv_results.add(new_row, ignore_index=True)

In [ ]:
grid_search_cv_results.to_csv("/content/drive/MyDrive/MLProject/Codes/Task2/Results/grid_search_cv_results_RF_"+model_from+"_model_"+dataset_name+".csv",index=False)

In [ ]:
with open('/content/drive/MyDrive/MLProject/Codes/Task2/Results/grid_search_results_RF_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Hyperparameters\tTrain_Accuracy\tTest_Accuracy\n")
    for params, train_acc, test_acc in zip(grid_search.cv_results_['params'], train_accuracy, test_accuracy):
        file.write(f"{params}\t{train_acc:.4f}\t{test_acc:.4f}\n")

In [ ]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

In [ ]:
best_model = grid_search.best_estimator_z
pickle.dump(best_model, open('/content/drive/MyDrive/MLProject/Codes/Task2/Models/RF_best_model_config_'+model_from+'_model_'+dataset_name+'.sav', 'wb'))

In [ ]:
#loaded_model = pickle.load(open('/content/drive/MyDrive/MLProject/Codes/Task2/Models/RF_best_model_config_'+model_from+'_model_'+dataset_name+'.sav', 'rb'))

In [ ]:
print(best_model)

In [ ]:
# Assess the performance on training data
y_train_pred = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_conf_matrix = confusion_matrix(y_train, y_train_pred)
train_classification_rep = classification_report(y_train, y_train_pred)

# Print training results
print("Training Data Results:")
print("Accuracy:", train_accuracy)
print("Confusion Matrix:")
print(train_conf_matrix)
print("Classification Report:")
print(train_classification_rep)

In [ ]:
with open('/content/drive/MyDrive/MLProject/Codes/Task2/Results/training_results_Best_RF_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Training Data Results:\n")
    file.write(f"Accuracy: {train_accuracy}\n")
    file.write("Confusion Matrix:\n")
    file.write(f"{train_conf_matrix}\n")
    file.write("Classification Report:\n")
    file.write(f"{train_classification_rep}\n")

In [ ]:
# Assess the performance on testing data
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_conf_matrix = confusion_matrix(y_test, y_test_pred)
test_classification_rep = classification_report(y_test, y_test_pred)

# Print testing results
print("\nTesting Data Results:")
print("Accuracy:", test_accuracy)
print("Confusion Matrix:")
print(test_conf_matrix)
print("Classification Report:")
print(test_classification_rep)

In [ ]:
with open('/content/drive/MyDrive/MLProject/Data/Task2/Results/testing_results_Best_RF_'+model_from+'_model_'+dataset_name+'.txt', 'w') as file:
    file.write("Testing Data Results:\n")
    file.write(f"Accuracy: {test_accuracy}\n")
    file.write("Confusion Matrix:\n")
    file.write(f"{test_conf_matrix}\n")
    file.write("Classification Report:\n")
    file.write(f"{test_classification_rep}\n")